In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark


ModuleNotFoundError: No module named 'findspark'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python dataframe example").getOrCreate()

In [10]:
customer = spark.read.load("/home/hduser/Downloads/sharedfolder/customers.txt", format = "csv", sep = "\t" , inferSchema = "true", header = "true")


In [11]:
customer.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [12]:
customer.show(5)

+-----------+--------------+-------------+--------------+----------------+
|customer_id| customer_name|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-------------+--------------+----------------+
|      11039|   Mary Torres|       Caguas|            PR|             725|
|       5623|    Jose Haley|     Columbus|            OH|           43207|
|       5829|    Mary Smith|      Houston|            TX|           77015|
|       6336|Richard Maddox|       Caguas|            PR|             725|
|       1708|Margaret Booth|    Arlington|            TX|           76010|
+-----------+--------------+-------------+--------------+----------------+
only showing top 5 rows



In [17]:
customer.select(customer['customer_name'] ,customer['customer_city']).show(5)

+--------------+-------------+
| customer_name|customer_city|
+--------------+-------------+
|   Mary Torres|       Caguas|
|    Jose Haley|     Columbus|
|    Mary Smith|      Houston|
|Richard Maddox|       Caguas|
|Margaret Booth|    Arlington|
+--------------+-------------+
only showing top 5 rows



In [18]:
customer.createTempView("cusomers")

In [24]:
sqldf = spark.sql("select * from cusomers where customer_state ='CA' ")

In [25]:
sqldf.show(5)

+-----------+----------------+---------------+--------------+----------------+
|customer_id|   customer_name|  customer_city|customer_state|customer_zipcode|
+-----------+----------------+---------------+--------------+----------------+
|       5577|      Mary Smith|        Modesto|            CA|           95350|
|       1745|      Mary Smith|Rowland Heights|            CA|           91748|
|      11444|Kathleen Patrick|      San Diego|            CA|           92109|
|       8846|    Thomas Smith|          Indio|            CA|           92201|
|       6237|  Bobby Anderson|       El Cajon|            CA|           92020|
+-----------+----------------+---------------+--------------+----------------+
only showing top 5 rows



In [31]:
sqlcount = spark.sql("select count(customer_state) as count,customer_state from cusomers group by customer_state")
sqlcount.show()

+-----+--------------+
|count|customer_state|
+-----+--------------+
|   19|            AZ|
|    2|            SC|
|    7|            LA|
|    1|            MN|
|   19|            NJ|
|    4|            DC|
|    4|            OR|
|   14|            VA|
|    2|            RI|
|    1|            KY|
|   28|            MI|
|   16|            NV|
|    9|            WI|
|    2|            ID|
|  187|            CA|
|    8|            CT|
|   19|            NC|
|   19|            MD|
|    1|            DE|
|   13|            MO|
+-----+--------------+
only showing top 20 rows



In [27]:
customer.groupBy("customer_state").count().show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AZ|   19|
|            SC|    2|
|            LA|    7|
|            MN|    1|
|            NJ|   19|
|            DC|    4|
|            OR|    4|
|            VA|   14|
|            RI|    2|
|            KY|    1|
|            MI|   28|
|            NV|   16|
|            WI|    9|
|            ID|    2|
|            CA|  187|
|            CT|    8|
|            NC|   19|
|            MD|   19|
|            DE|    1|
|            MO|   13|
+--------------+-----+
only showing top 20 rows



In [65]:
cstate = spark.sql("select  customer_state,count(customer_state) as cnt from cusomers group by customer_state having cnt > 50  ")

In [66]:
cstate.coalesce(1).write.parquet("/home/hduser/Desktop/cstate")

In [53]:
productDF = spark.read.load("/home/hduser/Downloads/sharedfolder/products.json", format = "json")

In [54]:
productDF.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: long (nullable = true)
 |-- salestxn_id: long (nullable = true)



In [68]:
productDF.createTempView("prod")

In [70]:
spark.sql("select * from prod").show(5)

+-----------+-----------+----------------+--------------------+-------------+----------------+-----------+
|category_id|customer_id|product_category|        product_name|product_price|product_quantity|salestxn_id|
+-----------+-----------+----------------+--------------------+-------------+----------------+-----------+
|         45|      12080|         Fishing|O'Brien Men's Neo...|        49.98|               2|      98660|
|         45|      12279|         Fishing|O'Brien Men's Neo...|        49.98|               2|      97749|
|         12|       3343|    Boxing & MMA|Under Armour Wome...|        31.99|               4|     103889|
|         45|      12382|         Fishing|O'Brien Men's Neo...|        49.98|               5|      22751|
|         47|       5032|         Boating|Pelican Sunstream...|       199.99|               1|     129001|
+-----------+-----------+----------------+--------------------+-------------+----------------+-----------+
only showing top 5 rows



In [71]:
spark.sql("select product_name,product_category,product_price from prod where product_price > 200  ").show()

+--------------------+------------------+-------------+
|        product_name|  product_category|product_price|
+--------------------+------------------+-------------+
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Diamondback Women...| Bike & Skate Shop|       299.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Field & Stream Sp...|Hunting & Shooting|       399.98|
|Field & Stream Sp...|Hunting & Shooting|       

In [76]:
spark.sql("select product_category, count(product_category) cnt from prod group by product_category order by cnt ASC  ").show()

+-------------------+---+
|   product_category|cnt|
+-------------------+---+
|Baseball & Softball|  4|
|  Training by Sport|  5|
|         Golf Shoes|  6|
|           Football| 13|
|   Cardio Equipment| 14|
|   Men's Golf Clubs| 21|
|      Shop By Sport| 26|
|           Lacrosse| 32|
|  Strength Training| 35|
|         Basketball| 36|
|   Camping & Hiking| 44|
|Fitness Accessories| 47|
|        Electronics| 48|
|       Golf Apparel| 51|
|             Soccer| 53|
| Women's Golf Clubs| 57|
|   Tennis & Racquet| 66|
|  Golf Bags & Carts| 89|
|      Boys' Apparel|102|
|        Accessories|110|
+-------------------+---+
only showing top 20 rows

